In [3]:
import pandas as pd
import numpy as np
import re
import ast
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, linear_kernel
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
import warnings
warnings.filterwarnings('ignore')

In [4]:
# Load data
df = pd.read_csv('data/vnexpress_foods_detail_merged.csv')
print(f"Dataset shape: {df.shape}")
df.head()

Dataset shape: (893, 13)


,link,type_of_food,title,description,author_name,cook_time,num_of_people,calories,num_of_ingredients,ingredients,step,note,post_date
0,https://vnexpress.net/doi-song-cooking-cach-mu...,Món Tết,Cách muối dưa hành truyền thống,Dưa hành muối là món ăn truyền thống ngày Tết ...,Bùi Thủy,45 phút,8-10 người,459 kcal,5.0,"['1 kg hành củ tươi', 'Tro bếp hoặc nước vo gọ...",['Bước 1: Chọn hành củ: Nên chọn hành củ ta bá...,[],"Thứ năm, 23/1/2025, 16:32 (GMT+7)"
1,https://vnexpress.net/doi-song-cooking-su-hao-...,Món Tết,Su hào xào mực - món cổ Tết Bát Tràng,Đĩa xào khô ráo với su hào giòn ngọt quyện với...,Bùi Thủy,50 phút,4 - 5 người,1.162 kcal,6.0,"['2 củ su hào non', '1 con mực khô', '1/2 củ c...",['Bước 1: Chọn và sơ chế mực: Người dân làng g...,['Su hào xào mực cùng với canh măng mực là hai...,"Thứ sáu, 17/1/2025, 17:24 (GMT+7)"
2,https://vnexpress.net/doi-song-cooking-canh-ma...,Món Tết,Canh măng ngày Tết cổ truyền Hà Nội,"Măng ngấu vị, giòn ngon, móng giò hầm vừa độ s...",Bùi Thủy,100 phút,8 - 10 người,4.930 kcal,6.0,"['800 gr măng khô', '2 móng giò lợn', 'Nước dù...","['Bước 1: Chọn măng khô: Theo lối cũ, người nộ...",['Nếu tận dụng nước luộc gà nấu canh măng thì ...,"Thứ tư, 15/1/2025, 19:00 (GMT+7)"
3,https://vnexpress.net/doi-song-cooking-gia-han...,Món Tết,Giả hạnh nhân - món ngon Tết xưa Hà Nội,Đây là món ăn cổ truyền thường thấy trong cỗ T...,Bùi Thủy,60 phút,4-5 người,1.112 kcal,8.0,"['2 bộ lòng mề gà', '100 gr lạc', '50 gr hạt đ...",['Bước 1: Chọn và sơ chế lạc: Chọn lạc khô chắ...,['Hạnh nhân xào (hay giả hạnh nhân) là món ăn ...,"Thứ ba, 7/1/2025, 17:31 (GMT+7)"
4,https://vnexpress.net/doi-song-cooking-cha-bi-...,Món Tết,Chả bì ớt xiêm xanh,"Chả bì bóng đẹp, gói đều tay. Khi ăn vị ngọt m...",Bùi Thủy,60 phút,5-6 người,2.512 kcal,6.0,"['500 gr giò sống', '300 gr bì lợn', '20 - 30 ...","['Bước 1: Chọn và sơ chế bì lợn, chuẩn bị giò ...",['Nên sơ chế kỹ bì lợn để chả được thơm. Tùy t...,"Thứ ba, 31/12/2024, 18:00 (GMT+7)"


In [5]:
# Xem thông tin dữ liệu
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 893 entries, 0 to 892
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   link                893 non-null    object 
 1   type_of_food        893 non-null    object 
 2   title               893 non-null    object 
 3   description         893 non-null    object 
 4   author_name         456 non-null    object 
 5   cook_time           442 non-null    object 
 6   num_of_people       443 non-null    object 
 7   calories            437 non-null    object 
 8   num_of_ingredients  891 non-null    float64
 9   ingredients         893 non-null    object 
 10  step                893 non-null    object 
 11  note                893 non-null    object 
 12  post_date           893 non-null    object 
dtypes: float64(1), object(12)
memory usage: 90.8+ KB


## 1. Data Preprocessing

In [6]:
# Tạo bản copy để xử lý
data = df.copy()

# Xử lý missing values
data['title'] = data['title'].fillna('')
data['description'] = data['description'].fillna('')
data['step'] = data['step'].fillna('[]')
data['ingredients'] = data['ingredients'].fillna('[]')
data['type_of_food'] = data['type_of_food'].fillna('Unknown')

print("Missing values after handling:")
print(data[['title', 'description', 'step', 'ingredients', 'type_of_food', 'calories', 'cook_time']].isnull().sum())

Missing values after handling:
title             0
description       0
step              0
ingredients       0
type_of_food      0
calories        456
cook_time       451
dtype: int64


In [7]:
def parse_list_string(s):
    """Parse string representation of list to actual list"""
    if pd.isna(s) or s == '[]':
        return []
    try:
        return ast.literal_eval(s)
    except:
        return []

def clean_text(text):
    """Clean and normalize text"""
    if pd.isna(text):
        return ''
    text = str(text).lower()
    # Remove special characters but keep Vietnamese characters
    text = re.sub(r'[^\w\s\u00C0-\u1EF9]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def parse_cook_time(time_str):
    """Parse cook time string to minutes"""
    if pd.isna(time_str):
        return np.nan
    time_str = str(time_str).lower()
    minutes = 0
    
    # Find hours
    hour_match = re.search(r'(\d+)\s*(?:giờ|h|hour)', time_str)
    if hour_match:
        minutes += int(hour_match.group(1)) * 60
    
    # Find minutes
    min_match = re.search(r'(\d+)\s*(?:phút|p|min|minute)', time_str)
    if min_match:
        minutes += int(min_match.group(1))
    
    # If only number found
    if minutes == 0:
        num_match = re.search(r'(\d+)', time_str)
        if num_match:
            minutes = int(num_match.group(1))
    
    return minutes if minutes > 0 else np.nan

def parse_calories(cal_str):
    """Parse calories string to numeric"""
    if pd.isna(cal_str):
        return np.nan
    cal_str = str(cal_str).replace('.', '').replace(',', '')
    match = re.search(r'(\d+)', cal_str)
    if match:
        return float(match.group(1))
    return np.nan

In [8]:
# Apply preprocessing
# Parse ingredients and steps
data['ingredients_list'] = data['ingredients'].apply(parse_list_string)
data['step_list'] = data['step'].apply(parse_list_string)

# Clean text fields
data['title_clean'] = data['title'].apply(clean_text)
data['description_clean'] = data['description'].apply(clean_text)
data['step_clean'] = data['step_list'].apply(lambda x: ' '.join([clean_text(s) for s in x]))

# Parse numeric fields
data['cook_time_minutes'] = data['cook_time'].apply(parse_cook_time)
data['calories_numeric'] = data['calories'].apply(parse_calories)

# Clean ingredients for Jaccard
data['ingredients_clean'] = data['ingredients_list'].apply(
    lambda x: set([clean_text(ing) for ing in x if ing])
)

print("Preprocessing completed!")
data[['title', 'title_clean', 'cook_time', 'cook_time_minutes', 'calories', 'calories_numeric']].head()

Preprocessing completed!


,title,title_clean,cook_time,cook_time_minutes,calories,calories_numeric
0,Cách muối dưa hành truyền thống,cách muối dưa hành truyền thống,45 phút,45.0,459 kcal,459.0
1,Su hào xào mực - món cổ Tết Bát Tràng,su hào xào mực món cổ tết bát tràng,50 phút,50.0,1.162 kcal,1162.0
2,Canh măng ngày Tết cổ truyền Hà Nội,canh măng ngày tết cổ truyền hà nội,100 phút,100.0,4.930 kcal,4930.0
3,Giả hạnh nhân - món ngon Tết xưa Hà Nội,giả hạnh nhân món ngon tết xưa hà nội,60 phút,60.0,1.112 kcal,1112.0
4,Chả bì ớt xiêm xanh,chả bì ớt xiêm xanh,60 phút,60.0,2.512 kcal,2512.0


In [9]:
# Kiểm tra kết quả preprocessing
print("Sample ingredients_clean:")
for i, ing in enumerate(data['ingredients_clean'].head(3)):
    print(f"\nRecipe {i+1}: {data['title'].iloc[i]}")
    print(f"Ingredients: {ing}")

Sample ingredients_clean:

Recipe 1: Cách muối dưa hành truyền thống
Ingredients: {'lọ sạch', 'tro bếp hoặc nước vo gọa', '1 kg hành củ tươi', 'muối hạt đường', 'cà rốt trang trí tùy chọn'}

Recipe 2: Su hào xào mực - món cổ Tết Bát Tràng
Ingredients: {'1 con mực khô', 'gia vị mắm muối đường hạt tiêu rượu trắng gừng', 'mỡ lợn hoặc dầu ăn', '1 2 củ cà rốt', 'rau mùi trang trí', '2 củ su hào non'}

Recipe 3: Canh măng ngày Tết cổ truyền Hà Nội
Ingredients: {'hành khô hành củ', 'gia vị nước mắm truyền thống muối', 'nước dùng gà hoặc ninh xương lợn', 'nước vo gạo ngâm măng', '800 gr măng khô', '2 móng giò lợn'}


## 2. TF-IDF Based Recommendation

Sử dụng TF-IDF vectorizer trên text content (title + description + steps) và tính cosine similarity

In [10]:
# Combine text features
data['combined_text'] = data['title_clean'] + ' ' + data['description_clean'] + ' ' + data['step_clean']

# TF-IDF Vectorization
tfidf_vectorizer = TfidfVectorizer(
    max_features=5000,
    ngram_range=(1, 2),  # unigrams and bigrams
    min_df=1,
    max_df=0.95,
    stop_words=None  # Keep Vietnamese words
)

tfidf_matrix = tfidf_vectorizer.fit_transform(data['combined_text'])
print(f"TF-IDF Matrix shape: {tfidf_matrix.shape}")

TF-IDF Matrix shape: (893, 5000)


In [11]:
# Compute TF-IDF cosine similarity matrix
tfidf_similarity = cosine_similarity(tfidf_matrix, tfidf_matrix)
print(f"TF-IDF Similarity Matrix shape: {tfidf_similarity.shape}")

TF-IDF Similarity Matrix shape: (893, 893)


In [12]:
def get_tfidf_recommendations(recipe_idx, similarity_matrix, df, top_n=5):
    """
    Get top N similar recipes based on TF-IDF similarity
    
    Parameters:
    - recipe_idx: Index of the recipe to find recommendations for
    - similarity_matrix: Precomputed similarity matrix
    - df: DataFrame containing recipe data
    - top_n: Number of recommendations to return
    
    Returns:
    - DataFrame with recommended recipes and similarity scores
    """
    # Get similarity scores for the given recipe
    sim_scores = list(enumerate(similarity_matrix[recipe_idx]))
    
    # Sort by similarity (descending)
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # Get top N (excluding the recipe itself)
    sim_scores = sim_scores[1:top_n+1]
    
    # Get recipe indices
    recipe_indices = [i[0] for i in sim_scores]
    scores = [i[1] for i in sim_scores]
    
    # Create result DataFrame
    result = df.iloc[recipe_indices][['title', 'type_of_food', 'calories', 'cook_time']].copy()
    result['similarity_score'] = scores
    
    return result

def recommend_by_title_tfidf(title, df, similarity_matrix, top_n=5):
    """
    Get recommendations by recipe title
    """
    # Find recipe index by title
    matches = df[df['title'].str.contains(title, case=False, na=False)]
    if len(matches) == 0:
        print(f"No recipe found with title containing: {title}")
        return None
    
    recipe_idx = matches.index[0]
    print(f"\n🍽️ Input Recipe: {df.loc[recipe_idx, 'title']}")
    print(f"   Type: {df.loc[recipe_idx, 'type_of_food']}")
    print(f"   Calories: {df.loc[recipe_idx, 'calories']}")
    print("\n📋 TF-IDF Recommendations:")
    
    return get_tfidf_recommendations(recipe_idx, similarity_matrix, df, top_n)

In [13]:
# Test TF-IDF Recommendation
print("METHOD 1: TF-IDF BASED RECOMMENDATION")

# Test với một món ăn
test_title = "Canh măng"
recommendations = recommend_by_title_tfidf(test_title, data, tfidf_similarity, top_n=5)
if recommendations is not None:
    display(recommendations)

METHOD 1: TF-IDF BASED RECOMMENDATION

🍽️ Input Recipe: Canh măng ngày Tết cổ truyền Hà Nội
   Type: Món Tết
   Calories: 4.930 kcal

📋 TF-IDF Recommendations:


,title,type_of_food,calories,cook_time,similarity_score
65,Canh móng giò hầm măng khô kiểu Bắc,Món Tết,2.390 kcal,90 phút,0.562129
155,Cách làm canh măng mực - đặc sản truyền thống ...,Món ngon hàng ngày,1.190 kcal,120 phút,0.455145
28,Cách làm thịt kho măng khô- món ngon Tết miền ...,Món Tết,3.043 kcal,75 phút,0.426037
330,Cách làm miến vịt măng khô đơn giản tại nhà,Món ngon hàng ngày,3.520 kcal,90 phút,0.423153
142,Cách làm miến măng vịt thơm ngon tại nhà,Món ngon hàng ngày,2.608 kcal,90 phút,0.392017


## 3. Ingredient-Based Recommendation (Jaccard Similarity)

Sử dụng Jaccard Similarity để so sánh sự tương đồng giữa các thành phần nguyên liệu

In [14]:
def jaccard_similarity(set1, set2):
    """
    Calculate Jaccard similarity between two sets
    J(A,B) = |A ∩ B| / |A ∪ B|
    """
    if len(set1) == 0 and len(set2) == 0:
        return 0.0
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    return intersection / union if union > 0 else 0.0

def compute_jaccard_similarity_matrix(ingredients_list):
    """
    Compute Jaccard similarity matrix for all recipes
    """
    n = len(ingredients_list)
    similarity_matrix = np.zeros((n, n))
    
    for i in range(n):
        for j in range(i, n):
            sim = jaccard_similarity(ingredients_list[i], ingredients_list[j])
            similarity_matrix[i][j] = sim
            similarity_matrix[j][i] = sim
    
    return similarity_matrix

In [15]:
# Compute Jaccard similarity matrix
print("Computing Jaccard similarity matrix...")
ingredients_sets = data['ingredients_clean'].tolist()
jaccard_similarity_matrix = compute_jaccard_similarity_matrix(ingredients_sets)
print(f"Jaccard Similarity Matrix shape: {jaccard_similarity_matrix.shape}")

Computing Jaccard similarity matrix...
Jaccard Similarity Matrix shape: (893, 893)


In [16]:
def get_ingredient_recommendations(recipe_idx, similarity_matrix, df, top_n=5):
    """
    Get top N similar recipes based on ingredient (Jaccard) similarity
    """
    sim_scores = list(enumerate(similarity_matrix[recipe_idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:top_n+1]
    
    recipe_indices = [i[0] for i in sim_scores]
    scores = [i[1] for i in sim_scores]
    
    result = df.iloc[recipe_indices][['title', 'type_of_food', 'calories', 'cook_time']].copy()
    result['jaccard_score'] = scores
    
    return result

def recommend_by_title_jaccard(title, df, similarity_matrix, top_n=5):
    """
    Get ingredient-based recommendations by recipe title
    """
    matches = df[df['title'].str.contains(title, case=False, na=False)]
    if len(matches) == 0:
        print(f"No recipe found with title containing: {title}")
        return None
    
    recipe_idx = matches.index[0]
    print(f"\n🍽️ Input Recipe: {df.loc[recipe_idx, 'title']}")
    print(f"   Ingredients: {list(df.loc[recipe_idx, 'ingredients_clean'])[:5]}...")
    print("\n📋 Ingredient-Based (Jaccard) Recommendations:")
    
    return get_ingredient_recommendations(recipe_idx, similarity_matrix, df, top_n)

In [17]:
# Test Ingredient-Based Recommendation
print("METHOD 2: INGREDIENT-BASED RECOMMENDATION (JACCARD)")

test_title = "Canh măng"
recommendations = recommend_by_title_jaccard(test_title, data, jaccard_similarity_matrix, top_n=5)
if recommendations is not None:
    display(recommendations)

METHOD 2: INGREDIENT-BASED RECOMMENDATION (JACCARD)

🍽️ Input Recipe: Canh măng ngày Tết cổ truyền Hà Nội
   Ingredients: ['hành khô hành củ', 'gia vị nước mắm truyền thống muối', 'nước dùng gà hoặc ninh xương lợn', 'nước vo gạo ngâm măng', '800 gr măng khô']...

📋 Ingredient-Based (Jaccard) Recommendations:


,title,type_of_food,calories,cook_time,jaccard_score
65,Canh móng giò hầm măng khô kiểu Bắc,Món Tết,2.390 kcal,90 phút,0.090909
0,Cách muối dưa hành truyền thống,Món Tết,459 kcal,45 phút,0.000000
1,Su hào xào mực - món cổ Tết Bát Tràng,Món Tết,1.162 kcal,50 phút,0.000000
3,Giả hạnh nhân - món ngon Tết xưa Hà Nội,Món Tết,1.112 kcal,60 phút,0.000000
4,Chả bì ớt xiêm xanh,Món Tết,2.512 kcal,60 phút,0.000000


## 4. Metadata Similarity

Tính toán similarity dựa trên metadata: calories, cook_time, type_of_food

In [18]:
def compute_metadata_similarity_matrix(df):
    """
    Compute similarity matrix based on metadata features:
    - calories (normalized)
    - cook_time (normalized)
    - type_of_food (one-hot encoded)
    """
    n = len(df)
    
    # Normalize numeric features
    scaler = MinMaxScaler()
    
    # Handle missing values
    calories = df['calories_numeric'].fillna(df['calories_numeric'].median()).values.reshape(-1, 1)
    cook_time = df['cook_time_minutes'].fillna(df['cook_time_minutes'].median()).values.reshape(-1, 1)
    
    calories_normalized = scaler.fit_transform(calories)
    cook_time_normalized = scaler.fit_transform(cook_time)
    
    # One-hot encode type_of_food
    type_dummies = pd.get_dummies(df['type_of_food'], prefix='type')
    
    # Combine all features
    metadata_features = np.hstack([
        calories_normalized,
        cook_time_normalized,
        type_dummies.values
    ])
    
    print(f"Metadata features shape: {metadata_features.shape}")
    
    # Compute cosine similarity
    similarity_matrix = cosine_similarity(metadata_features, metadata_features)
    
    return similarity_matrix, metadata_features

In [19]:
# Compute metadata similarity
print("Computing Metadata similarity matrix...")
metadata_similarity_matrix, metadata_features = compute_metadata_similarity_matrix(data)
print(f"Metadata Similarity Matrix shape: {metadata_similarity_matrix.shape}")

Computing Metadata similarity matrix...
Metadata features shape: (893, 14)
Metadata Similarity Matrix shape: (893, 893)


In [20]:
def get_metadata_recommendations(recipe_idx, similarity_matrix, df, top_n=5):
    """
    Get top N similar recipes based on metadata similarity
    """
    sim_scores = list(enumerate(similarity_matrix[recipe_idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:top_n+1]
    
    recipe_indices = [i[0] for i in sim_scores]
    scores = [i[1] for i in sim_scores]
    
    result = df.iloc[recipe_indices][['title', 'type_of_food', 'calories', 'cook_time']].copy()
    result['metadata_score'] = scores
    
    return result

def recommend_by_title_metadata(title, df, similarity_matrix, top_n=5):
    """
    Get metadata-based recommendations by recipe title
    """
    matches = df[df['title'].str.contains(title, case=False, na=False)]
    if len(matches) == 0:
        print(f"No recipe found with title containing: {title}")
        return None
    
    recipe_idx = matches.index[0]
    print(f"\n🍽️ Input Recipe: {df.loc[recipe_idx, 'title']}")
    print(f"   Type: {df.loc[recipe_idx, 'type_of_food']}")
    print(f"   Calories: {df.loc[recipe_idx, 'calories']}")
    print(f"   Cook time: {df.loc[recipe_idx, 'cook_time']}")
    print("\n📋 Metadata-Based Recommendations:")
    
    return get_metadata_recommendations(recipe_idx, similarity_matrix, df, top_n)

In [21]:
# Test Metadata-Based Recommendation
print("METADATA-BASED RECOMMENDATION")

test_title = "Canh măng"
recommendations = recommend_by_title_metadata(test_title, data, metadata_similarity_matrix, top_n=5)
if recommendations is not None:
    display(recommendations)

METADATA-BASED RECOMMENDATION

🍽️ Input Recipe: Canh măng ngày Tết cổ truyền Hà Nội
   Type: Món Tết
   Calories: 4.930 kcal
   Cook time: 100 phút

📋 Metadata-Based Recommendations:


,title,type_of_food,calories,cook_time,metadata_score
51,Cách làm mứt dừa truyền thống đơn giản,Món Tết,4.420 kcal,100 phút,0.999736
62,Mứt me gọi Tết miền Tây,Món Tết,4.000 kcal,120 phút,0.998716
77,Sườn tảng nướng cam cho tiệc Giáng sinh,Món Tết,3.738 kcal,90 phút,0.998460
73,Cách làm mứt vỏ cam đón Tết,Món Tết,3.400 kcal,90 phút,0.997503
56,"Hướng dẫn cách nấu xôi gấc ngon, dẻo, đẹp",Món Tết,3.849 kcal,60 phút,0.997341


## 5. Hybrid Approach (Weighted Combination)

Kết hợp 3 phương pháp với trọng số:
- Text Similarity (TF-IDF): **0.4**
- Ingredient Similarity (Jaccard): **0.4**
- Metadata Similarity: **0.2**

In [22]:
def compute_hybrid_similarity(tfidf_sim, jaccard_sim, metadata_sim, 
                              w_tfidf=0.4, w_jaccard=0.4, w_metadata=0.2):
    """
    Compute hybrid similarity matrix as weighted combination
    
    Parameters:
    - tfidf_sim: TF-IDF similarity matrix
    - jaccard_sim: Jaccard similarity matrix
    - metadata_sim: Metadata similarity matrix
    - w_tfidf: Weight for TF-IDF (default 0.4)
    - w_jaccard: Weight for Jaccard (default 0.4)
    - w_metadata: Weight for metadata (default 0.2)
    
    Returns:
    - Hybrid similarity matrix
    """
    # Ensure weights sum to 1
    total_weight = w_tfidf + w_jaccard + w_metadata
    w_tfidf /= total_weight
    w_jaccard /= total_weight
    w_metadata /= total_weight
    
    print(f"Weights: TF-IDF={w_tfidf:.2f}, Jaccard={w_jaccard:.2f}, Metadata={w_metadata:.2f}")
    
    hybrid_similarity = (w_tfidf * tfidf_sim + 
                        w_jaccard * jaccard_sim + 
                        w_metadata * metadata_sim)
    
    return hybrid_similarity

In [23]:
# Compute hybrid similarity matrix
print("Computing Hybrid similarity matrix...")
hybrid_similarity_matrix = compute_hybrid_similarity(
    tfidf_similarity, 
    jaccard_similarity_matrix, 
    metadata_similarity_matrix,
    w_tfidf=0.4,
    w_jaccard=0.4,
    w_metadata=0.2
)
print(f"Hybrid Similarity Matrix shape: {hybrid_similarity_matrix.shape}")

Computing Hybrid similarity matrix...
Weights: TF-IDF=0.40, Jaccard=0.40, Metadata=0.20
Hybrid Similarity Matrix shape: (893, 893)


In [24]:
def get_hybrid_recommendations(recipe_idx, hybrid_sim, tfidf_sim, jaccard_sim, metadata_sim, df, top_n=5):
    """
    Get top N similar recipes based on hybrid similarity with detailed scores
    """
    sim_scores = list(enumerate(hybrid_sim[recipe_idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:top_n+1]
    
    recipe_indices = [i[0] for i in sim_scores]
    hybrid_scores = [i[1] for i in sim_scores]
    
    result = df.iloc[recipe_indices][['title', 'type_of_food', 'calories', 'cook_time']].copy()
    result['hybrid_score'] = hybrid_scores
    result['tfidf_score'] = [tfidf_sim[recipe_idx][i] for i in recipe_indices]
    result['jaccard_score'] = [jaccard_sim[recipe_idx][i] for i in recipe_indices]
    result['metadata_score'] = [metadata_sim[recipe_idx][i] for i in recipe_indices]
    
    return result

def recommend_by_title_hybrid(title, df, hybrid_sim, tfidf_sim, jaccard_sim, metadata_sim, top_n=5):
    """
    Get hybrid recommendations by recipe title
    """
    matches = df[df['title'].str.contains(title, case=False, na=False)]
    if len(matches) == 0:
        print(f"No recipe found with title containing: {title}")
        return None
    
    recipe_idx = matches.index[0]
    print(f"\n🍽️ Input Recipe: {df.loc[recipe_idx, 'title']}")
    print(f"   Type: {df.loc[recipe_idx, 'type_of_food']}")
    print(f"   Calories: {df.loc[recipe_idx, 'calories']}")
    print(f"   Cook time: {df.loc[recipe_idx, 'cook_time']}")
    print(f"   Ingredients: {list(df.loc[recipe_idx, 'ingredients_clean'])[:3]}...")
    print("\n📋 Hybrid Recommendations (TF-IDF: 0.4 + Jaccard: 0.4 + Metadata: 0.2):")
    
    return get_hybrid_recommendations(recipe_idx, hybrid_sim, tfidf_sim, jaccard_sim, metadata_sim, df, top_n)

In [25]:
# Test Hybrid Recommendation
print("METHOD 3: HYBRID APPROACH (WEIGHTED COMBINATION)")

test_title = "Canh măng"
recommendations = recommend_by_title_hybrid(
    test_title, data, 
    hybrid_similarity_matrix, 
    tfidf_similarity, 
    jaccard_similarity_matrix, 
    metadata_similarity_matrix, 
    top_n=5
)
if recommendations is not None:
    display(recommendations)

METHOD 3: HYBRID APPROACH (WEIGHTED COMBINATION)

🍽️ Input Recipe: Canh măng ngày Tết cổ truyền Hà Nội
   Type: Món Tết
   Calories: 4.930 kcal
   Cook time: 100 phút
   Ingredients: ['hành khô hành củ', 'gia vị nước mắm truyền thống muối', 'nước dùng gà hoặc ninh xương lợn']...

📋 Hybrid Recommendations (TF-IDF: 0.4 + Jaccard: 0.4 + Metadata: 0.2):


,title,type_of_food,calories,cook_time,hybrid_score,tfidf_score,jaccard_score,metadata_score
65,Canh móng giò hầm măng khô kiểu Bắc,Món Tết,2.390 kcal,90 phút,0.459846,0.562129,0.090909,0.993154
28,Cách làm thịt kho măng khô- món ngon Tết miền ...,Món Tết,3.043 kcal,75 phút,0.369565,0.426037,0.000000,0.995752
13,Cách làm canh bóng thả Hà Nội nấu theo lối xưa,Món Tết,1.157 kcal,85 phút,0.295789,0.247144,0.000000,0.984655
16,Cách làm giò xào truyền thống kiểu Bắc,Món Tết,3.082 kcal,75 phút,0.284204,0.212557,0.000000,0.995905
20,Cách nấu thịt đông kiểu truyền thống,Món Tết,2.968 kcal,70 phút,0.274980,0.189854,0.000000,0.995194


## 6. Comparison of All Methods

In [26]:
def compare_all_methods(title, df, tfidf_sim, jaccard_sim, metadata_sim, hybrid_sim, top_n=5):
    """
    Compare recommendations from all methods for a given recipe
    """
    matches = df[df['title'].str.contains(title, case=False, na=False)]
    if len(matches) == 0:
        print(f"No recipe found with title containing: {title}")
        return None
    
    recipe_idx = matches.index[0]
    
    print(f"🍽️ COMPARING RECOMMENDATIONS FOR: {df.loc[recipe_idx, 'title']}")
    
    print(f"\n📌 Recipe Details:")
    print(f"   Type: {df.loc[recipe_idx, 'type_of_food']}")
    print(f"   Calories: {df.loc[recipe_idx, 'calories']}")
    print(f"   Cook time: {df.loc[recipe_idx, 'cook_time']}")
    
    # TF-IDF Recommendations
    print("📊 METHOD 1: TF-IDF Based")
    tfidf_recs = get_tfidf_recommendations(recipe_idx, tfidf_sim, df, top_n)
    display(tfidf_recs)
    
    # Jaccard Recommendations
    print("📊 METHOD 2: Ingredient-Based (Jaccard)")
    jaccard_recs = get_ingredient_recommendations(recipe_idx, jaccard_sim, df, top_n)
    display(jaccard_recs)
    
    # Hybrid Recommendations
    print("📊 METHOD 3: Hybrid (TF-IDF: 0.4 + Jaccard: 0.4 + Metadata: 0.2)")
    hybrid_recs = get_hybrid_recommendations(recipe_idx, hybrid_sim, tfidf_sim, jaccard_sim, metadata_sim, df, top_n)
    display(hybrid_recs)
    
    return {
        'tfidf': tfidf_recs,
        'jaccard': jaccard_recs,
        'hybrid': hybrid_recs
    }

In [27]:
# Compare all methods for a sample recipe
results = compare_all_methods(
    "Canh măng", data, 
    tfidf_similarity, 
    jaccard_similarity_matrix, 
    metadata_similarity_matrix,
    hybrid_similarity_matrix,
    top_n=5
)

🍽️ COMPARING RECOMMENDATIONS FOR: Canh măng ngày Tết cổ truyền Hà Nội

📌 Recipe Details:
   Type: Món Tết
   Calories: 4.930 kcal
   Cook time: 100 phút
📊 METHOD 1: TF-IDF Based


,title,type_of_food,calories,cook_time,similarity_score
65,Canh móng giò hầm măng khô kiểu Bắc,Món Tết,2.390 kcal,90 phút,0.562129
155,Cách làm canh măng mực - đặc sản truyền thống ...,Món ngon hàng ngày,1.190 kcal,120 phút,0.455145
28,Cách làm thịt kho măng khô- món ngon Tết miền ...,Món Tết,3.043 kcal,75 phút,0.426037
330,Cách làm miến vịt măng khô đơn giản tại nhà,Món ngon hàng ngày,3.520 kcal,90 phút,0.423153
142,Cách làm miến măng vịt thơm ngon tại nhà,Món ngon hàng ngày,2.608 kcal,90 phút,0.392017


📊 METHOD 2: Ingredient-Based (Jaccard)


,title,type_of_food,calories,cook_time,jaccard_score
65,Canh móng giò hầm măng khô kiểu Bắc,Món Tết,2.390 kcal,90 phút,0.090909
0,Cách muối dưa hành truyền thống,Món Tết,459 kcal,45 phút,0.000000
1,Su hào xào mực - món cổ Tết Bát Tràng,Món Tết,1.162 kcal,50 phút,0.000000
3,Giả hạnh nhân - món ngon Tết xưa Hà Nội,Món Tết,1.112 kcal,60 phút,0.000000
4,Chả bì ớt xiêm xanh,Món Tết,2.512 kcal,60 phút,0.000000


📊 METHOD 3: Hybrid (TF-IDF: 0.4 + Jaccard: 0.4 + Metadata: 0.2)


,title,type_of_food,calories,cook_time,hybrid_score,tfidf_score,jaccard_score,metadata_score
65,Canh móng giò hầm măng khô kiểu Bắc,Món Tết,2.390 kcal,90 phút,0.459846,0.562129,0.090909,0.993154
28,Cách làm thịt kho măng khô- món ngon Tết miền ...,Món Tết,3.043 kcal,75 phút,0.369565,0.426037,0.000000,0.995752
13,Cách làm canh bóng thả Hà Nội nấu theo lối xưa,Món Tết,1.157 kcal,85 phút,0.295789,0.247144,0.000000,0.984655
16,Cách làm giò xào truyền thống kiểu Bắc,Món Tết,3.082 kcal,75 phút,0.284204,0.212557,0.000000,0.995905
20,Cách nấu thịt đông kiểu truyền thống,Món Tết,2.968 kcal,70 phút,0.274980,0.189854,0.000000,0.995194


In [28]:
# Test với một món ăn khác
results2 = compare_all_methods(
    "Nem", data, 
    tfidf_similarity, 
    jaccard_similarity_matrix, 
    metadata_similarity_matrix,
    hybrid_similarity_matrix,
    top_n=5
)

🍽️ COMPARING RECOMMENDATIONS FOR: Cách làm nem rán kiểu truyền thống miền Bắc

📌 Recipe Details:
   Type: Món Tết
   Calories: 2.517 kcal
   Cook time: 65 phút
📊 METHOD 1: TF-IDF Based


,title,type_of_food,calories,cook_time,similarity_score
54,Công thức và Cách làm nem rán Hà Nội chuẩn nhất,Món Tết,2.615 kcal,75 phút,0.695238
188,Cách làm nem chim bồ câu đúng chuẩn,Món ngon hàng ngày,2.175 kcal,80 phút,0.604944
713,"Cách làm nem cá rô giòn, ngon đổi vị ngày hè",Thực đơn cho ngày nắng nóng,2.227 kcal,70 phút,0.595163
36,Cách làm nem hải sản giòn rụm cho ngày Tết,Món Tết,1.938 kcal,80 phút,0.557951
440,"Cùng làm món Nem ốc - Thơm ngon, Hấp dẫn và Độ...",Món ngon hàng ngày,1.508 kcal,90 phút,0.535166


📊 METHOD 2: Ingredient-Based (Jaccard)


,title,type_of_food,calories,cook_time,jaccard_score
54,Công thức và Cách làm nem rán Hà Nội chuẩn nhất,Món Tết,2.615 kcal,75 phút,0.153846
36,Cách làm nem hải sản giòn rụm cho ngày Tết,Món Tết,1.938 kcal,80 phút,0.142857
21,Cách làm miến xào thập cẩm ngon mà dễ làm,Món Tết,1.356 kcal,30 phút,0.136364
145,Bí quyết làm miến xào hải sản không bị dính,Món ngon hàng ngày,1.366 kcal,30 phút,0.130435
213,Thịt gà nấu đông - món ngon miền Bắc,Món ngon hàng ngày,1.439 kcal,90 phút,0.117647


📊 METHOD 3: Hybrid (TF-IDF: 0.4 + Jaccard: 0.4 + Metadata: 0.2)


,title,type_of_food,calories,cook_time,hybrid_score,tfidf_score,jaccard_score,metadata_score
54,Công thức và Cách làm nem rán Hà Nội chuẩn nhất,Món Tết,2.615 kcal,75 phút,0.539612,0.695238,0.153846,0.999893
36,Cách làm nem hải sản giòn rụm cho ngày Tết,Món Tết,1.938 kcal,80 phút,0.480205,0.557951,0.142857,0.999406
21,Cách làm miến xào thập cẩm ngon mà dễ làm,Món Tết,1.356 kcal,30 phút,0.335764,0.204392,0.136364,0.997307
26,Cách làm hạnh nhân xào - vị Tết Hà Nội xưa,Món Tết,1.323 kcal,35 phút,0.313895,0.190729,0.095238,0.997540
11,Cách làm miến xào lòng mề gà,Món Tết,1.405 kcal,45 phút,0.307074,0.177655,0.090909,0.998242


## 7. Complete Recommendation Class

In [29]:
class FoodRecommender:
    """
    Content-Based Food Recommendation System
    
    Methods:
    1. TF-IDF Based: Uses title + description + steps
    2. Ingredient Based: Uses Jaccard similarity on ingredients
    3. Hybrid: Weighted combination of TF-IDF (0.4) + Jaccard (0.4) + Metadata (0.2)
    """
    
    def __init__(self, df):
        self.df = df.copy()
        self._preprocess()
        self._build_similarity_matrices()
    
    def _preprocess(self):
        """Preprocess the data"""
        # Parse lists
        self.df['ingredients_list'] = self.df['ingredients'].apply(parse_list_string)
        self.df['step_list'] = self.df['step'].apply(parse_list_string)
        
        # Clean text
        self.df['title_clean'] = self.df['title'].apply(clean_text)
        self.df['description_clean'] = self.df['description'].fillna('').apply(clean_text)
        self.df['step_clean'] = self.df['step_list'].apply(lambda x: ' '.join([clean_text(s) for s in x]))
        self.df['combined_text'] = self.df['title_clean'] + ' ' + self.df['description_clean'] + ' ' + self.df['step_clean']
        
        # Parse numeric
        self.df['cook_time_minutes'] = self.df['cook_time'].apply(parse_cook_time)
        self.df['calories_numeric'] = self.df['calories'].apply(parse_calories)
        
        # Ingredients set
        self.df['ingredients_clean'] = self.df['ingredients_list'].apply(
            lambda x: set([clean_text(ing) for ing in x if ing])
        )
        
        print("✅ Data preprocessing completed!")
    
    def _build_similarity_matrices(self):
        """Build all similarity matrices"""
        # TF-IDF
        print("Building TF-IDF similarity matrix...")
        tfidf_vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1, 2), min_df=1, max_df=0.95)
        tfidf_matrix = tfidf_vectorizer.fit_transform(self.df['combined_text'])
        self.tfidf_sim = cosine_similarity(tfidf_matrix)
        
        # Jaccard
        print("Building Jaccard similarity matrix...")
        self.jaccard_sim = compute_jaccard_similarity_matrix(self.df['ingredients_clean'].tolist())
        
        # Metadata
        print("Building Metadata similarity matrix...")
        self.metadata_sim, _ = compute_metadata_similarity_matrix(self.df)
        
        # Hybrid
        print("Building Hybrid similarity matrix...")
        self.hybrid_sim = compute_hybrid_similarity(
            self.tfidf_sim, self.jaccard_sim, self.metadata_sim,
            w_tfidf=0.4, w_jaccard=0.4, w_metadata=0.2
        )
        
        print("✅ All similarity matrices built!")
    
    def recommend(self, title, method='hybrid', top_n=5):
        """
        Get recommendations for a recipe
        
        Parameters:
        - title: Recipe title (partial match)
        - method: 'tfidf', 'jaccard', or 'hybrid'
        - top_n: Number of recommendations
        
        Returns:
        - DataFrame with recommendations
        """
        matches = self.df[self.df['title'].str.contains(title, case=False, na=False)]
        if len(matches) == 0:
            print(f"No recipe found with title containing: {title}")
            return None
        
        recipe_idx = matches.index[0]
        
        print(f"\n🍽️ Input Recipe: {self.df.loc[recipe_idx, 'title']}")
        
        if method == 'tfidf':
            return get_tfidf_recommendations(recipe_idx, self.tfidf_sim, self.df, top_n)
        elif method == 'jaccard':
            return get_ingredient_recommendations(recipe_idx, self.jaccard_sim, self.df, top_n)
        else:  # hybrid
            return get_hybrid_recommendations(
                recipe_idx, self.hybrid_sim, self.tfidf_sim, 
                self.jaccard_sim, self.metadata_sim, self.df, top_n
            )
    
    def recommend_by_index(self, idx, method='hybrid', top_n=5):
        """
        Get recommendations by recipe index
        """
        if idx < 0 or idx >= len(self.df):
            print(f"Invalid index: {idx}")
            return None
        
        print(f"\n🍽️ Input Recipe: {self.df.iloc[idx]['title']}")
        
        if method == 'tfidf':
            return get_tfidf_recommendations(idx, self.tfidf_sim, self.df, top_n)
        elif method == 'jaccard':
            return get_ingredient_recommendations(idx, self.jaccard_sim, self.df, top_n)
        else:  # hybrid
            return get_hybrid_recommendations(
                idx, self.hybrid_sim, self.tfidf_sim,
                self.jaccard_sim, self.metadata_sim, self.df, top_n
            )

In [30]:
# Create recommender instance
print("Initializing Food Recommender...")
recommender = FoodRecommender(df)

Initializing Food Recommender...
✅ Data preprocessing completed!
Building TF-IDF similarity matrix...
Building Jaccard similarity matrix...
Building Metadata similarity matrix...
Metadata features shape: (893, 14)
Building Hybrid similarity matrix...
Weights: TF-IDF=0.40, Jaccard=0.40, Metadata=0.20
✅ All similarity matrices built!


In [31]:
# Test the recommender
print("TESTING FOOD RECOMMENDER")

# TF-IDF method
print("\n📊 TF-IDF Method:")
display(recommender.recommend("Thịt kho", method='tfidf', top_n=5))

# Jaccard method
print("\n📊 Jaccard Method:")
display(recommender.recommend("Thịt kho", method='jaccard', top_n=5))

# Hybrid method
print("\n📊 Hybrid Method:")
display(recommender.recommend("Thịt kho", method='hybrid', top_n=5))

TESTING FOOD RECOMMENDER

📊 TF-IDF Method:

🍽️ Input Recipe: Cách làm thịt kho hột vịt truyền thống miền Tây


,title,type_of_food,calories,cook_time,similarity_score
46,Thịt kho tàu - món ăn đặc trưng Tết Nam Bộ,Món Tết,3.998 kcal,30 phút,0.542763
57,Thịt kho tàu kiểu Bắc – món ngon Tết đến,Món Tết,2.270 kcal,60 phút,0.384554
144,Cách làm thịt ba chỉ kho trứng cút đậm đà,Món ngon hàng ngày,1.908 kcal,50 phút,0.352585
28,Cách làm thịt kho măng khô- món ngon Tết miền ...,Món Tết,3.043 kcal,75 phút,0.345311
445,"Công thức món thịt kho trứng, cùi dừa THƠM NGO...",Món ngon hàng ngày,NaN,NaN,0.336558



📊 Jaccard Method:

🍽️ Input Recipe: Cách làm thịt kho hột vịt truyền thống miền Tây


,title,type_of_food,calories,cook_time,jaccard_score
392,Cách làm thịt kho măng đơn giản mà siêu ngon t...,Món ngon hàng ngày,2.034 kcal,50 phút,0.100000
144,Cách làm thịt ba chỉ kho trứng cút đậm đà,Món ngon hàng ngày,1.908 kcal,50 phút,0.083333
0,Cách muối dưa hành truyền thống,Món Tết,459 kcal,45 phút,0.000000
1,Su hào xào mực - món cổ Tết Bát Tràng,Món Tết,1.162 kcal,50 phút,0.000000
2,Canh măng ngày Tết cổ truyền Hà Nội,Món Tết,4.930 kcal,100 phút,0.000000



📊 Hybrid Method:

🍽️ Input Recipe: Cách làm thịt kho hột vịt truyền thống miền Tây


,title,type_of_food,calories,cook_time,hybrid_score,tfidf_score,jaccard_score,metadata_score
46,Thịt kho tàu - món ăn đặc trưng Tết Nam Bộ,Món Tết,3.998 kcal,30 phút,0.415752,0.542763,0.0,0.993231
57,Thịt kho tàu kiểu Bắc – món ngon Tết đến,Món Tết,2.270 kcal,60 phút,0.353484,0.384554,0.0,0.998313
28,Cách làm thịt kho măng khô- món ngon Tết miền ...,Món Tết,3.043 kcal,75 phút,0.337967,0.345311,0.0,0.999212
50,Cá trắm đen kho riềng - món ngon Tết xưa Hà Nội,Món Tết,1.792 kcal,240 phút,0.320866,0.311360,0.0,0.981607
20,Cách nấu thịt đông kiểu truyền thống,Món Tết,2.968 kcal,70 phút,0.294377,0.236439,0.0,0.999006


## 8. Summary

### Các phương pháp đã implement:

1. **TF-IDF Based Recommendation**
   - Sử dụng text vectorization với TF-IDF
   - Features: title + description + cooking steps
   - Similarity: Cosine similarity

2. **Ingredient-Based Recommendation**
   - So sánh danh sách nguyên liệu
   - Similarity: Jaccard similarity
   - $J(A,B) = \frac{|A \cap B|}{|A \cup B|}$

3. **Hybrid Approach**
   - Weighted combination của các phương pháp
   - Weights: TF-IDF (0.4) + Jaccard (0.4) + Metadata (0.2)
   - Metadata bao gồm: calories, cook_time, type_of_food

### Next Steps:
- Evaluation metrics (Precision, Recall, NDCG)
- A/B Testing
- User feedback integration
- Deep learning approaches (Embeddings, Neural Collaborative Filtering)